In [1]:
import numpy as np

In [5]:
import os

# Directory containing the npz files
directory = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/internal/dump_match_pairs'

# List all files in the directory
files = os.listdir(directory)

# Filter out the npz files
npz_files = [file for file in files if file.endswith('.npz')]

# Sort the files to get the first one
npz_files.sort()

# Load the first npz file
first_npz_file = npz_files[0]
data = np.load(os.path.join(directory, first_npz_file))

# Print the keys of the npz file
print(data.files)

['keypoints0', 'keypoints1', 'matches', 'match_confidence', 'scene_name']


In [12]:
import cv2
import pandas as pd

# Iterate over all npz files
# Initialize lists to store the results
samples = []
fundamental_matrices = []

# Iterate over all npz files
for npz_file in npz_files:
    # Load the npz file
    data = np.load(os.path.join(directory, npz_file))
    
    # Extract matches and keypoints
    matches = data['matches']
    kpts0 = data['keypoints0']
    kpts1 = data['keypoints1']
    
    # Filter valid matches
    valid = matches > -1
    mkpts0 = kpts0[valid]
    mkpts1 = kpts1[matches[valid]]
    
    # Estimate the fundamental matrix using the matched keypoints
    F, mask = cv2.findFundamentalMat(mkpts0, mkpts1, cv2.FM_RANSAC)
    
    # Extract scene name and image names
    scene_name = data['scene_name']
    image_name0 = npz_file.split('-')[0]
    image_name1 = npz_file.split('-')[1]
    
    # Append the results to the lists
    samples.append(f"{scene_name};{image_name0}-{image_name1}")
    F = F.reshape(-1)
    fundamental_matrices.append(F)

# Create a dataframe with the collected data
df = pd.DataFrame({
    'sample_id': samples,
    'fundamental_matrix': fundamental_matrices
})

print(df)

                                           sample_id  \
0  st_peters_square;61838905_5998204213-31114085_...   
1  st_peters_square;61888375_5977160696-31114085_...   
2  st_peters_square;99669631_2524741025-73248999_...   

                                  fundamental_matrix  
0  [-4.23302886104571e-07, -3.130600047119747e-05...  
1  [-4.399606690758423e-06, -0.000241455609844919...  
2  [1.3851692080129176e-07, 3.669134139240682e-06...  


In [13]:
df['fundamental_matrix'] = df['fundamental_matrix'].apply(lambda x: ' '.join(f'{num:.5e}' for num in x))


In [14]:
df

,sample_id,fundamental_matrix
0,st_peters_square;61838905_5998204213-31114085_...,-4.23303e-07 -3.13060e-05 -5.08634e-03 3.26446...
1,st_peters_square;61888375_5977160696-31114085_...,-4.39961e-06 -2.41456e-04 5.72142e-02 2.62679e...
2,st_peters_square;99669631_2524741025-73248999_...,1.38517e-07 3.66913e-06 5.15050e-04 -3.89309e-...
